# Classification Models

## Table of contents:
* [Maximum Likelihood logistic regression](#maxlike)
* [Bayesian logistic regression](#bayesian)
* [Non-linear logistic regression](#non-linear)
* [Dual logistic regression](#dual)
* [Relevance vector classification](#relevant-vector)
* [Incremental fitting and boosting](#boosting)
* [Classification trees](#trees)
* [Multi-class logistic regression](#multiclass)
* [Random trees, forests, and ferns](#forest)
* [Multi-class logistic regression](#multiclass)

# Dual logistic regression

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spsp
import scipy.stats as spst
%matplotlib inline

# usual gangs

In [ ]:
def generate_data_set_dim1_nonlinear(ns):
    """
    this function generates a 1d data set of two classes which are linearly separated.
    
    Arguments:
        ns: number of samples
    
    Output:
        ds1   : data on the negative side (2, ns). ds1[1] = 0 is the label.
        ds2   : data on the positive side (2, ns). ds2[1] = 1 is the label.
        data  : data combined ds1 and ds2 and shuffled. (1, ns)
        label : label combined ds1 and ds2 and shuffled. (1, ns)
        datab : data combined ds1 and ds2 and shuffled. data1[0] = 1 for bias. (2, ns)
    """
    ns_neg1 = int(0.3 * ns)
    ns_neg2 = int(0.2 * ns)
    ns_neg  = ns_neg1 + ns_neg2
    ns_pos1 = int(0.3 * ns)
    ns_pos2 = ns - ns_neg1 - ns_neg2 - ns_pos1
    ns_pos  = ns_pos1 + ns_pos2
    mus = [-6, 2, 6, -2]
    
    dt_neg1 = np.random.standard_t(3, size=ns_neg1) + mus[0]
    dt_neg2 = np.random.standard_t(3, size=ns_neg2) + mus[1]
    ds1 = np.hstack((dt_neg1, dt_neg2))
    ds1 = np.vstack((ds1, np.zeros(ns_neg)))
    
    dt_pos1 = np.random.standard_t(3, size=ns_pos1) + mus[2]
    dt_pos2 = np.random.standard_t(3, size=ns_pos2) + mus[3]
    ds2 = np.hstack((dt_pos1, dt_pos2))
    ds2 = np.vstack((ds2, np.ones(ns_pos)))
    
    ds = np.hstack((ds1, ds2))
    np.random.shuffle(ds.T)
    data = ds[0]
    label = ds[1][np.newaxis, :]
    datab = np.vstack((np.ones(ns), data))
    
    plt.rcParams['figure.figsize'] = (5.0, 3.0)
    plt.plot(ds1[0], ds1[1], 'r.', markersize=3)
    plt.plot(ds2[0], ds2[1], 'b.', markersize=3)
    return data, label, datab


def generate_data_set_dim2_linear(ns):
    """
    this function generates a 2d data set of two classes which are linearly separated.
    
    Arguments:
        ns: number of samples
    
    Output:
        ds1: data on the negative side (3, ns). ds1[1] = 0 is the label.
        ds2: data on the positive side (3, ns). ds2[1] = 0 is the label.
        data  : data combined ds1 and ds2 and shuffled. (2, ns)
        label : label combined ds1 and ds2 and shuffled. (2, ns)
        datab : data combined ds1 and ds2 and shuffled. datab[0] = 1 for bias. (3, ns)
    """
    ns_neg = int(0.4 * ns)
    ns_pos = int(0.4 * ns)
    ns_pen = ns - ns_neg - ns_pos
    nfar_neg = int(0.7 * ns_neg)
    nbdr_neg = ns_neg - nfar_neg
    nfar_pos = int(0.7 * ns_pos)
    nbdr_pos = ns_pos - nfar_pos
    
    far_neg = 8 * (np.random.rand(nfar_neg) - .5) - 5
    bdr_neg = np.random.standard_t(4, size=nbdr_neg) - 0.5
    ds1x = np.hstack((far_neg, bdr_neg, bdr_pen))
    ds1y = 10 * (np.random.rand(ns_neg) - .5)
    ds1 = np.vstack((ds1x, ds1y))
    
    far_pos = 8 * (np.random.rand(nfar_pos) - .5) + 5
    bdr_pos = np.random.standard_t(4, size=nbdr_pos) + 0.5
    ds2x = np.hstack((far_pos, bdr_pos))
    ds2y = 10 * (np.random.rand(ns_pos) - .5)
    ds2 = np.vstack((ds2x, ds2y))
    
    ang = np.pi * np.random.rand() 
    c, s = np.cos(ang), np.sin(ang)
    rot = np.array([[c, s], [-s, c]])
    
    ds1 = np.dot(rot, ds1)
    ds2 = np.dot(rot, ds2)
    
    ds1 = np.vstack((ds1, np.zeros(ns_neg)))
    ds2 = np.vstack((ds2, np.ones(ns_pos)))
    ds = np.hstack((ds1, ds2))
    np.random.shuffle(ds.T)
    
    data = ds[:2]
    label = ds[2][np.newaxis, :]
    datab = np.vstack((np.ones(ns), data))
    
    #plt.rcParams['figure.figsize'] = (5.0, 5.0)
    #plt.plot(ds1[0], ds1[1], 'r.', markersize=3)
    #plt.plot(ds2[0], ds2[1], 'b.', markersize=3)
    return ds1, ds2, data, label, datab, ang


def cost_function(z, w, phi):
    """
    this function computes logistic cost function
    
    Arguments:
        z   :  (K + 1, N)  1 is for bias.
        w   :  (1, N)
        phi :  (1, K + 1)
    
    Output:
        L:     scalar. cost function.
    """
    sig = sigmoid(z, phi)
    wlogsig1 = np.dot(w, np.log(sig))
    wlogsig2 = np.dot(1 - w, np.log(1 - sig))
    L = wlogsig1 + wlogsig2
    return L

    
def sigmoid(z, phi):
    """
    this function computes sigmoid function
    
    Arguments:
        z   :  (K + 1, N)  1 is for bias.
        phi :  (1, K + 1)
    
    Output:
        sig  :  (N, 1)
    """
    phix = np.dot(phi, z)
    denom = 1 + np.exp(-phix) 
    sig = 1 / denom
    return sig.T


def da_dphi(z):
    """
    this function generates da/dphi.
    
    Arguments:
        z: generated bases including bias at the zeroth row. ((K+1), N).
    
    Output:
        z: da/dphi. ((K+1), N)
    """
    return z


def da_dalp(z, x, phi, alp, lmd):
    """
    this function generates da/dalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1), N).
        x  : original cooridnates of the samples. (D, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.
    
    Intermediate:
        pf    : (K+1, N)
        pfx   : (K*D, N)
        xx    : (K*D, N)
        alpx  : (K*D, 1)

    Output:
        da/dalp: (K*D, N)
    """
    D = x.shape[0]
    K = alp.shape[1]
    N = x.shape[1]
    pf = phi.T * z / lmd
    pfx = np.kron(pf[1:], np.ones((D, 1)))
    xx = np.tile(x, (K, 1))
    alpx = np.reshape(alp.T, (D*K, 1))
    dadalp = pfx * (xx - alpx)
    return dadalp


def da_dtheta(z, x, phi, alp, lmd):
    """
    this function generates da/dtheta.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.

    Output:
        da/dtheta: ((K+1+KD), N)
    """
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    dadtht = np.zeros((K1 + K*D, N))
    dadtht[:K1, :] = da_dphi(z)
    dadtht[K1:, :] = da_dalp(z, x, phi, alp, lmd)
    return dadtht


def dda_dphi_dphi(z):
    """
    this function generates dda/dphidphi.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).

    Output:
        dda/dphidphi: ((K+1), (K+1))
    """
    K1 = z.shape[0]
    N  = z.shape[1]
    ddadphidphi = np.zeros((K1, K1, N))
    return ddadphidphi


def dda_dphi_dalp(z, x, phi, alp, lmd):
    """
    this function generates dda/dphidalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.
    
    Intermediate:
        pf    : (K+1, N)
        pfx   : (K*D, N)
        xx    : (K*D, N)
        alpx  : (K*D, 1)
        dlt   : (K*D, N)

    Output:
        ddadphidalp: (KD, K, N)
    """
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    pf = z/lmd
    pfx = np.kron(pf[1:], np.ones((D, 1)))
    xx = np.tile(x, (K, 1))
    alpx = alp.reshape(D*K, 1)
    dlt = pfx * (xx - alpx)
    ddadphidalp = np.tile(dlt[:, np.newaxis, :], (1, K, 1))
    return ddadphidalp


def dda_dalp_dalp(z, x, phi, alp, lmd):
    """
    this function generates dda/dphidalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.
    
    Intermediate:
        pf    : (K+1, N)
        pfx   : (K*D, N)
        xx    : (K*D, N)
        alpx  : (K*D, 1)
        dlt   : (K*D, N)

    Output:
        da/dtheta: ((K+1+KD), N)
    """
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    pf = phi.T * z / lmd
    xx = np.tile(x[:, np.newaxis, :], (1, K, 1))
    alpx = np.tile(alp[:, :, np.newaxis], (D, 1, 1))
    dlt = xx - alpx
    dltx = dlt[np.newaxis, :, :, :]
    dltxT = dlt[:, np.newaxis, :, :]
    pare = (dltxT * dltx)/lmd
    pare -= np.eye(D)[:, :, np.newaxis,  np.newaxis]
    dda_dpda = np.zeros((K*D, K*D, N))
    for k in range(K):
        dda_dpda[D*k:D*(k+1), D*k:D*(k+1), :] = pare[:, :, k, :] * pf[k, :]
    return dda_dpda


def dda_dtheta_dtheta(z, x, phi, alp, lmd):
    """
    this function generates dda/dphidalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.

    Output:
        da/dtheta: ((K+1+KD), N)
    """
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    nn = K1 + K*D
    ddadtdt = np.zeros((nn, nn, N))
    ddadtdt[:K1, :K1, :] = dda_dphi_dphi(z)
    ddadpda = dda_dphi_dalp(z, x, phi, alp, lmd)
    ddadtdt[K1:, 1:K1, :] = ddadpda
    ddadtdt[1:K1, K1:, :] = np.swapaxes(ddadpda, 0, 1)
    ddadtdt[K1:, K1:,  :] = dda_dalp_dalp(z, x, phi, alp, lmd)
    return ddadtdt


def dL_dtheta(z, x, w, phi, alp, lmd):
    """
    this function generates dda/dphidalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        w  : label. (1, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.
    
    Intermediate:
        sig   : (N, 1)
        dadt  : ((K+1+KD), N)
        dlt   : (1, N)

    Output:
        dLdt: ((K+1+KD), 1)
    """
    dadt = da_dtheta(z, x, phi, alp, lmd)
    sig = sigmoid(z, phi)
    dLdt = np.dot(dadt, w.T - sig)
    dLdt *= -1
    return dLdt


def ddL_dtheta_dtheta(z, x, w, phi, alp, lmd):
    """
    this function generates dda/dphidalp.
    
    Arguments:
        z  : generated bases including bias at the zeroth row. ((K+1) * N).
        x  : original cooridnates of the samples. (D, N)
        w  : label. (1, N)
        phi: phi. (1, (K+1))
        alp: alpha. (D, K)
        lmd: scalar.
    
    Intermediate:
        sig   : (N, 1)
        dadt  : ((K+1+KD), N)
        dlt   : (1, N)
        err   : (N, )

    Output:
        ddLdtdt: ((K+1+KD), (K+1+KD))
    """
    dadt = da_dtheta(z, x, phi, alp, lmd)
    dadts = dadt[np.newaxis, :, :] * dadt[:, np.newaxis, :]
    ddadtdt = dda_dtheta_dtheta(z, x, phi, alp, lmd)
    sig = sigmoid(z, phi)
    err = (w - sig.T).squeeze()
    sigsig1 = (sig * (sig - 1)).squeeze()
    ddLdtdt = np.einsum('i,jki->jk', sigsig1, dadts)
    ddLdtdt -= np.einsum('i,jki->jk', err, ddadtdt)
    ddLdtdt *= -1
    return ddLdtdt


def newton_method_update(z, x, w, phi, alp, beta, lmd):
    """
    this function computes sigmoid function
    
    Arguments:
        z    : generated bases including bias at the zeroth row. ((K+1) * N).
        x    : original cooridnates of the samples. (D, N)
        phi  : phi. (1, (K+1))
        alp  : alpha. (D, K)
        beta : scalar. learning rate.
        lmd  : scalar.
    
    Intermediate:
        dLdt   : ((K+1+KD), 1)
        ddLdtdt: ((K+1+KD), (K+1+KD))
        
    Output:
        theta: (1, (K+1+KD))
    """
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    theta = np.hstack((phi, alp.T.reshape(1, K*D)))
    dLdt    = dL_dtheta(z, x, w, phi, alp, lmd)
    ddLdtdt = ddL_dtheta_dtheta(z, x, w, phi, alp, lmd)
    ddLdtdt_inv = np.linalg.inv(ddLdtdt)
    theta -= beta * np.dot(ddLdtdt_inv, dLdt).T
    phi = theta[0, :K1].reshape(1, K1)
    alp = theta[0, K1:].reshape(K, D).T
    return phi, alp
    
    
def newton_method(z, x, w, phi, alp, beta, lmd):
    """
    this function computes sigmoid function
    
    Arguments:
        z    : generated bases including bias at the zeroth row. ((K+1) * N).
        x    : original cooridnates of the samples. (D, N)
        phi  : phi. (1, (K+1))
        alp  : alpha. (D, K)
        beta : scalar. learning rate.
        lmd  : scalar.
        
    Output:
        theta: (1, (K+1+KD))
    """
    import copy
    D = x.shape[0]
    K = alp.shape[1]
    K1 = z.shape[0]
    N = x.shape[1]
    nn = K1 + K * D
    dlt = 1
    step = 0
    thetas = np.zeros((10000, nn))
    theta = np.hstack((phi, alp.T.reshape(1, K*D)))
    thetas[step, :] = theta
    while dlt > 0.001:
        step += 1
        temp = copy.deepcopy(theta)
        phi, alp = newton_method_update(z, x, w, phi, alp, beta, lmd)       
        theta = np.hstack((phi, alp.T.reshape(1, K*D)))
        thetas[step, :] = theta
        dlt = np.max(np.abs(temp - theta)/temp)
    strng = '{:d} steps done: \n'.format(step + 1)
    for i in range(nn):
        strng += '{:1.4f}, '.format(theta.squeeze()[i])
    print(strng)
    thetas = thetas[:(step + 1), :]
    phi = theta[0, :K1].reshape(1, K1)
    alp = theta[0, K1:].reshape(K, D).T
    return phi, alp, thetas, thetas


def maximum_a_posterior(datab, label, phi, alpha, sgp):
    """
    this function computes sigmoid function
    
    Arguments:
        datab:   (D, ns)  D contains bias as well.
        label:   (1, ns)
        phi  :   (1, D)
        alpha:   scalar
        sgp  :   scalar. prior sigma
    
    Output:
        mu_map   :  (D, 1). MAP phi.
        sgms_map :  (D, D). MAP sigma squared.
    """
    phi, _ = newton_method(datab, label, phi, alpha, sgp)
    mu_map = phi
    hess = dL_hess(datab, phi, sgp)
    sgms_map = -np.linalg.inv(hess)
    return mu_map, sgms_map


def bayesian_inference(mu, sgms, nres):
    """
    Arguments:
        mu:    (1, D). MAP phi
        sgma:  (D, D). MAP sigma squared
        nres:  resolution of the curve/image
        
    Output:
        plot.
    """
    D = mu.shape[1]
    if D == 3:
        nn = nres**2
        xres = np.linspace(-10, 10, nres)
        yres = np.linspace(-10, 10, nres)
        yg, xg = np.meshgrid(yres, xres)
        x_flat = xg.reshape(1, nn)
        y_flat = yg.reshape(1, nn)
        x1_flat = np.vstack((np.ones(nn), x_flat))
        xy1_flat = np.vstack((x1_flat, y_flat))
        mus  = np.dot(phi, xy1_flat)
        sgmss = (np.dot(sgms, xy1_flat) * xy1_flat).sum(axis=0)
        arg = np.sqrt(1 + np.pi * sgmss / 8)
        denom = 1 + np.exp(-mus / arg)
        lmbd = 1 / denom
        lmbd = lmbd.reshape((nres, nres)).T
        lmbd = np.fliplr(lmbd)
        return lmbd
    elif D == 2:
        xres = np.linspace(-10, 10, nres)
        x1_flat = np.vstack((np.ones(nres), xres))
        mus  = np.dot(phi, x1_flat)
        sgmss = (np.dot(sgms, x1_flat) * x1_flat).sum(axis=0)
        arg = np.sqrt(1 + np.pi * sgmss / 8)
        denom = 1 + np.exp(-mus / arg)
        lmbd = 1 / denom
        return lmbd
    else:
        print('We don''t cover such case here')


def radial_basis(x, ng, lmd):
    """
    Arguments:
        x:    (1, nx). nx number of x coordinates. could be grid, could be samples.
        ng:   scalar. number of gaussians to use. 
        lmd:  scalar. std of the gaussian.
        
    Output:
        bases: (D, nx).
    """
    D = ng + 1
    nx = x.shape[0]
    mus = 12 * (np.random.rand(ng) - 0.5)
    bases = np.ones((D, nx))
    for i in range(1, D):
        bases[i, :] = spst.norm.pdf(x, mus[i - 1], lmd)
    return bases


def plot_decision_boundary_dim1(phi, alp, lmd):
    """
    this function computes sigmoid function
    
    Arguments:
        phi  : phi. (1, (K+1))
        alp  : alpha. (D, K)
        beta : scalar. learning rate.
        lmd  : scalar.
    
    Intermediate:
        dLdt   : ((K+1+KD), 1)
        ddLdtdt: ((K+1+KD), (K+1+KD))
        
    Output:
        theta: (1, (K+1+KD))
    """
    D = alp.shape[0]
    K = alp.shape[1]
    K1 = phi.shape[1]
    nx = 201
    x = np.linspace(-10, 10, nx)
    z = np.ones((K1, nx))
    for i in range(1, K1):
        z[i, :] = spst.norm.pdf(x, alp[:, i-1].squeeze(), lmd)
    dbdr = sigmoid(z, phi)
    return x, dbdr
